In [0]:
#Load and Transform Pipeline

#1-mount the data lakes
#2-loads csvs into the landing data lake
#3-convert csvs to parquet and move then to processing data lake
#4-create sql database
#5-create tables based on parquet format files
#6-specific analysis wil be moved to curated data lake and then loaded into sql tables
#7-powerbi application reads directly from sql tables at databrick rest api service


# Mounting Data lakes

In [0]:
dbutils.fs.unmount("/mnt/landing") 

/mnt/landing has been unmounted.
Out[85]: True

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "eb1abf3d-a317-4bd1-9f1f-4624aa42f72a",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="olistsecretscope",key="olistsecretvault"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/78bf4db5-3661-49fa-9cd8-053d3e4f1a00/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://landing@olistdl.dfs.core.windows.net/",
  mount_point = "/mnt/landing",
  extra_configs = configs)

Out[86]: True

In [0]:
dbutils.fs.ls("/mnt/landing")

Out[87]: [FileInfo(path='dbfs:/mnt/landing/olist_customers_dataset.csv', name='olist_customers_dataset.csv', size=9457240),
 FileInfo(path='dbfs:/mnt/landing/olist_geolocation_dataset.csv', name='olist_geolocation_dataset.csv', size=70275331),
 FileInfo(path='dbfs:/mnt/landing/olist_order_items_dataset.csv', name='olist_order_items_dataset.csv', size=16471076),
 FileInfo(path='dbfs:/mnt/landing/olist_order_payments_dataset.csv', name='olist_order_payments_dataset.csv', size=6063136),
 FileInfo(path='dbfs:/mnt/landing/olist_order_reviews_dataset.csv', name='olist_order_reviews_dataset.csv', size=16110569),
 FileInfo(path='dbfs:/mnt/landing/olist_orders_dataset.csv', name='olist_orders_dataset.csv', size=22041348),
 FileInfo(path='dbfs:/mnt/landing/olist_sellers_dataset.csv', name='olist_sellers_dataset.csv', size=192469),
 FileInfo(path='dbfs:/mnt/landing/product_category_name_translation.csv', name='product_category_name_translation.csv', size=2896)]

In [0]:
dbutils.fs.unmount("/mnt/processing")

/mnt/processing has been unmounted.
Out[88]: True

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "eb1abf3d-a317-4bd1-9f1f-4624aa42f72a",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="olistsecretscope",key="olistsecretvault"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/78bf4db5-3661-49fa-9cd8-053d3e4f1a00/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://processing@olistdl.dfs.core.windows.net/",
  mount_point = "/mnt/processing",
  extra_configs = configs)

Out[89]: True

In [0]:
dbutils.fs.ls("/mnt/processing")

Out[90]: [FileInfo(path='dbfs:/mnt/processing/customers.parquet/', name='customers.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/customers_RJ.parquet/', name='customers_RJ.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/geolocation.parquet/', name='geolocation.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/order_items.parquet/', name='order_items.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/order_payments.parquet/', name='order_payments.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/order_reviews.parquet/', name='order_reviews.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/orders.parquet/', name='orders.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/product_category_name_translation.parquet/', name='product_category_name_translation.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/processing/sellers.parquet/', name='sellers.parquet/', size=0)]

In [0]:
dbutils.fs.unmount("/mnt/curated")

/mnt/curated has been unmounted.
Out[91]: True

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "eb1abf3d-a317-4bd1-9f1f-4624aa42f72a",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="olistsecretscope",key="olistsecretvault"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/78bf4db5-3661-49fa-9cd8-053d3e4f1a00/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://curated@olistdl.dfs.core.windows.net/",
  mount_point = "/mnt/curated",
  extra_configs = configs)

Out[92]: True

In [0]:
dbutils.fs.ls("/mnt/curated")

Out[93]: [FileInfo(path='dbfs:/mnt/curated/customers_RJ.csv/', name='customers_RJ.csv/', size=0)]

# Readings CSVs in Landing Data Lake to DataFrames

In [0]:
#read customer csv to dataframe and test it
df_customers = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_customers_dataset.csv")
 
#display the dataframe
display(df_customers)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,sao paulo,SP
0004164d20a9e969af783496f3408652,104bdb7e6a6cdceaa88c3ea5fa6b2b93,13272,valinhos,SP
000419c5494106c306a97b5635748086,14843983d4a159080f6afe4b7f346e7c,24220,niteroi,RJ
00046a560d407e99b969756e0b10f282,0b5295fc9819d831f68eb0e9a3e13ab7,20540,rio de janeiro,RJ
00050bf6e01e69d5c0fd612f1bcfb69c,e3cf594a99e810f58af53ed4820f25e5,98700,ijui,RS
000598caf2ef4117407665ac33275130,7e0516b486e92ed3f3afdd6d1276cfbd,35540,oliveira,MG


In [0]:
df_customers.printSchema()

root
-- customer_id: string (nullable = true)
-- customer_unique_id: string (nullable = true)
-- customer_zip_code_prefix: integer (nullable = true)
-- customer_city: string (nullable = true)
-- customer_state: string (nullable = true)

In [0]:
#read the rest of csv files to the respectives dataframes
df_geolocation = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_geolocation_dataset.csv")
df_order_items = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_order_items_dataset.csv")
df_order_payments = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_order_payments_dataset.csv")
df_order_reviews = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_order_reviews_dataset.csv")
df_orders = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_orders_dataset.csv")
df_sellers = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/olist_sellers_dataset.csv")
df_product_category_name_translation = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/landing/product_category_name_translation.csv")


# Create SQL Temp Views

In [0]:
df_customers.createOrReplaceTempView("customers_tempview")


In [0]:
%sql
SELECT *
FROM customers_tempview

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,sao paulo,SP
0004164d20a9e969af783496f3408652,104bdb7e6a6cdceaa88c3ea5fa6b2b93,13272,valinhos,SP
000419c5494106c306a97b5635748086,14843983d4a159080f6afe4b7f346e7c,24220,niteroi,RJ
00046a560d407e99b969756e0b10f282,0b5295fc9819d831f68eb0e9a3e13ab7,20540,rio de janeiro,RJ
00050bf6e01e69d5c0fd612f1bcfb69c,e3cf594a99e810f58af53ed4820f25e5,98700,ijui,RS
000598caf2ef4117407665ac33275130,7e0516b486e92ed3f3afdd6d1276cfbd,35540,oliveira,MG


Create SQL Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS customers_db

# Create SQL Tables

In [0]:
%sql
DROP TABLE IF EXISTS customers_db.customers

 -- geolocation
 -- order_items
 -- order_payments
 -- order_reviews
 -- orders
 -- sellers
 -- product_category_name_translation


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.geolocation


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.order_items


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.order_payments


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.order_reviews


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.orders


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.sellers


In [0]:
%sql
DROP TABLE IF EXISTS customers_db.product_category_name_translation


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.customers 
USING CSV
LOCATION '/mnt/landing/olist_customers_dataset.csv'
OPTIONS (header "true", inferSchema "true")




In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.geolocation 
USING CSV
LOCATION '/mnt/landing/olist_geolocation_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.order_items 
USING CSV
LOCATION '/mnt/landing/olist_order_items_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.order_payments 
USING CSV
LOCATION '/mnt/landing/olist_order_payments_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.order_reviews 
USING CSV
LOCATION '/mnt/landing/olist_order_reviews_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.orders 
USING CSV
LOCATION '/mnt/landing/olist_orders_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.sellers 
USING CSV
LOCATION '/mnt/landing/olist_sellers_dataset.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customers_db.product_category_name_translation 
USING CSV
LOCATION '/mnt/landing/product_category_name_translation.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
SELECT *
FROM customers_db.customers

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,sao paulo,SP
0004164d20a9e969af783496f3408652,104bdb7e6a6cdceaa88c3ea5fa6b2b93,13272,valinhos,SP
000419c5494106c306a97b5635748086,14843983d4a159080f6afe4b7f346e7c,24220,niteroi,RJ
00046a560d407e99b969756e0b10f282,0b5295fc9819d831f68eb0e9a3e13ab7,20540,rio de janeiro,RJ
00050bf6e01e69d5c0fd612f1bcfb69c,e3cf594a99e810f58af53ed4820f25e5,98700,ijui,RS
000598caf2ef4117407665ac33275130,7e0516b486e92ed3f3afdd6d1276cfbd,35540,oliveira,MG


In [0]:
%sql
DESCRIBE customers_db.customers

col_name,data_type,comment
customer_id,string,null
customer_unique_id,string,null
customer_zip_code_prefix,int,null
customer_city,string,null
customer_state,string,null


In [0]:
df_customers_SQL = spark.table('customers_db.customers')
display(df_customers_SQL)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,sao paulo,SP
0004164d20a9e969af783496f3408652,104bdb7e6a6cdceaa88c3ea5fa6b2b93,13272,valinhos,SP
000419c5494106c306a97b5635748086,14843983d4a159080f6afe4b7f346e7c,24220,niteroi,RJ
00046a560d407e99b969756e0b10f282,0b5295fc9819d831f68eb0e9a3e13ab7,20540,rio de janeiro,RJ
00050bf6e01e69d5c0fd612f1bcfb69c,e3cf594a99e810f58af53ed4820f25e5,98700,ijui,RS
000598caf2ef4117407665ac33275130,7e0516b486e92ed3f3afdd6d1276cfbd,35540,oliveira,MG


# Filtering the DataSet

In [0]:
df_customers_SQL.select('customer_state').distinct().show()

+--------------+
customer_state|
+--------------+
 SC|
 RO|
 PI|
 AM|
 RR|
 GO|
 TO|
 MT|
 SP|
 ES|
 PB|
 RS|
 MS|
 AL|
 MG|
 PA|
 BA|
 SE|
 PE|
 CE|
+--------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import col
df_customers_SQL = df_customers_SQL.filter(col("customer_state") == "RJ")

In [0]:
df_customers_SQL.select('customer_state').distinct().show()

+--------------+
customer_state|
+--------------+
 RJ|
+--------------+

# Write Full Parquet Datasets to Processing Data lake

In [0]:
df_customers.write.mode("overwrite").parquet("/mnt/processing/customers.parquet")
df_geolocation.write.mode("overwrite").parquet("/mnt/processing/geolocation.parquet")
df_order_items.write.mode("overwrite").parquet("/mnt/processing/order_items.parquet")
df_order_payments.write.mode("overwrite").parquet("/mnt/processing/order_payments.parquet")
df_order_reviews.write.mode("overwrite").parquet("/mnt/processing/order_reviews.parquet")
df_orders.write.mode("overwrite").parquet("/mnt/processing/orders.parquet")
df_sellers.write.mode("overwrite").parquet("/mnt/processing/sellers.parquet")
df_product_category_name_translation.write.mode("overwrite").parquet("/mnt/processing/product_category_name_translation.parquet")


# Write Filtered Parquet to Processing Data Lake

In [0]:
df_customers_SQL.write.mode("overwrite").parquet("/mnt/processing/customers_RJ.parquet")


In [0]:
df_customers_parq = spark.read.parquet("/mnt/processing/customers_RJ.parquet")
display(df_customers_parq)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


In [0]:
df_customers_parq.createOrReplaceTempView("CustomersParquetTable")
custparkSQL = spark.sql("select * from CustomersParquetTable where customer_state = 'RJ'")
display(custparkSQL)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


In [0]:
#df_customers.write.partitionBy("customer_state").parquet("/mnt/processing/customers_by_state.parquet")

# Create SQL Tables based on Parquet files at Processing Data Lake

In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.customers_pqt USING PARQUET OPTIONS (path "/mnt/processing/customers.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.geolocation_pqt USING PARQUET OPTIONS (path "/mnt/processing/geolocation.parquet", header "true", inferSchema "true")



In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.order_items_pqt USING PARQUET OPTIONS (path "/mnt/processing/order_items.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.order_payments_pqt USING PARQUET OPTIONS (path "/mnt/processing/order_payments.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.order_reviews_pqt USING PARQUET OPTIONS (path "/mnt/processing/order_reviews.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.orders_pqt USING PARQUET OPTIONS (path "/mnt/processing/orders.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.sellers_pqt USING PARQUET OPTIONS (path "/mnt/processing/sellers.parquet", header "true", inferSchema "true")


In [0]:
%sql
-- Full parquet
CREATE TABLE IF NOT EXISTS customers_db.product_category_name_translation_pqt USING PARQUET OPTIONS (path "/mnt/processing/product_category_name_translation.parquet", header "true", inferSchema "true")


In [0]:

%sql
-- Filtered parquet
CREATE TABLE IF NOT EXISTS customers_db.customers_RJ_pqt USING PARQUET OPTIONS (path "/mnt/processing/customers_RJ.parquet", header "true", inferSchema "true")


In [0]:
%sql
REFRESH TABLE customers_db.customers_RJ_pqt

In [0]:
%sql
SELECT * from customers_db.customers_RJ_pqt

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


In [0]:
df_customers_parq = spark.read.parquet("/mnt/processing/customers_RJ.parquet")
df_customers_parq.createOrReplaceTempView("CustomersParquetTableByState")
df_customers_by_state_parq = spark.sql("select * from CustomersParquetTableByState where customer_state='RJ'")
display(df_customers_by_state_parq)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


In [0]:
display(df_customers_parq)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


# Write processed CSVs to Curated Data Lake

In [0]:
df_customers_parq.write.option("header",True).option("delimiter",",").mode("overwrite").csv("/mnt/curated/customers_RJ.csv")

# Test Reading CSV file located at Curated Data Lake

In [0]:
#read in the data to dataframe df
df_RJ = spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load("/mnt/curated/customers_RJ.csv")
 
#display the dataframe
display(df_RJ)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ


In [0]:
%sql
-- Filtered Curated CSV
CREATE TABLE IF NOT EXISTS customers_db.customers_RJ_csv 
USING CSV
LOCATION '/mnt/curated/customers_RJ.csv'
OPTIONS (header "true", inferSchema "true")


In [0]:
%sql
REFRESH TABLE customers_db.customers_RJ_csv

In [0]:
%sql
Select * from customers_db.customers_RJ_csv 

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
72108dc3b6db1e78f3d9dd7b7b95266d,f7ba21d6b0af7a0cb8f9291a1d8e81a3,28660,bom jardim,RJ
7211f6a374836aa064addb1e52aa5500,a78856b0e5b47a539274777a789acd6c,24120,niteroi,RJ
72139f6cbf524640d2427b29fc084c15,47d1439acaabc312edcfc92dbab81e87,26215,nova iguacu,RJ
7214cb9a9251ade0ba75fda90e285b58,b2c10a20dc7be6c0df3715ad93805bdd,22260,rio de janeiro,RJ
7222d9a0c96d9a2813e905c54fa170c0,7936c03612607b0b78d0fb1cffd870b2,23914,angra dos reis,RJ
7224ce4000539b85758bdae0e06fb611,1c266d9dd6ac729c3b734f15714dcb11,28991,saquarema,RJ
722faefed2a13f6521b5b1af3aa04087,2708339a3f5355f56983737d54fbce59,22780,rio de janeiro,RJ
723a5aaedfd28127b7f4bf4ead493c51,5b9b3022d82b12bdc1e52267db0b0f79,20560,rio de janeiro,RJ
7241aae1af5ffcdb5411a9d23be378ea,0cd7d20504975ef3cd878aaffa80e482,20241,rio de janeiro,RJ
724a8db62950dcbfcbec9be37795ba1c,9bc1d14c71ab8184a43e64cfeb070d33,25780,sao jose do vale do rio preto,RJ
